In [2]:
!pip install folium pandas

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [6]:
# Download and read the `spacex_launch_geo.csv`
import requests
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = requests.get(URL)
spacex_csv_file = io.BytesIO(resp.content)
spacex_df=pd.read_csv(spacex_csv_file)

In [7]:
# Select relevant columns from the DataFrame
launch_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# Get the mean coordinates to center the map
mean_lat = launch_sites_df['Lat'].mean()
mean_long = launch_sites_df['Long'].mean()

# Create a map centered around the mean coordinates
site_map = folium.Map(location=[mean_lat, mean_long], zoom_start=5)

# Add a marker for each launch site
for index, site in launch_sites_df.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site'],
    ).add_to(site_map)

# Display the map
display(site_map)

In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [11]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [15]:
# Add a circle to each launch point on the map
for index, record in spacex_df.iterrows():
    # Determine the color based on the class value
    if record['class'] == 1:
        color = 'green'
    else:
        color = 'red'

    # Create a Circle object
    circle = folium.Circle(
        location=[record['Lat'], record['Long']],
        radius=50, # radius in meters
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.6,
        popup=f"Launch Site: {record['Launch Site']}, Outcome: {'Success' if record['class'] == 1 else 'Failure'}"
    )

    # Add the circle to the map
    site_map.add_child(circle)

# Display the map
display(site_map)

In [16]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [17]:
marker_cluster = MarkerCluster()

In [20]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [23]:
# Task 3: Calculate the distances between a launch site to its proximities

# Define the coordinates for a launch site (e.g., CCAFS LC-40)
launch_site_lat = 28.562302
launch_site_long = -80.577356
launch_site_coordinates = [launch_site_lat, launch_site_long]

# Define approximate coordinates for the equator and coastline
equator_coordinates = [0, launch_site_long] # Equator at the same longitude as the launch site
coastline_coordinates = [launch_site_lat, -80.5] # Approximate coastline near the launch site

# You can add more proximity points like roads, railways, or cities here
# For example:
# road_coordinates = [latitude, longitude]
# city_coordinates = [latitude, longitude]

In [25]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Calculate distance to the equator
distance_to_equator = calculate_distance(launch_site_lat, launch_site_long, equator_coordinates[0], equator_coordinates[1])
print(f"Distance to Equator: {distance_to_equator:.2f} km")

# Calculate distance to the coastline
distance_to_coastline = calculate_distance(launch_site_lat, launch_site_long, coastline_coordinates[0], coastline_coordinates[1])
print(f"Distance to Coastline: {distance_to_coastline:.2f} km")

Distance to Equator: 3176.98 km
Distance to Coastline: 7.56 km


In [26]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [28]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [30]:
# find coordinate of the closest coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Approximate coastline longitude near CCAFS LC-40
coastline_long_approx = -80.57163
coastline_lat = launch_site_lat # Assume coastline is at the same latitude as the launch site for simplicity

# Calculate the distance to the approximated coastline point
distance_coastline = calculate_distance(launch_site_lat, launch_site_long, coastline_lat, coastline_long_approx)

print(f"Approximate distance to coastline: {distance_coastline:.2f} km")

# Add a marker for the approximated coastline point on the map
coastline_marker_approx = folium.Marker(
    [coastline_lat, coastline_long_approx],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Closest Coastline (Approx.)',
        )
    )
site_map.add_child(coastline_marker_approx)

# Add a line from the launch site to the approximated coastline point
coastline_line_approx = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_long], [coastline_lat, coastline_long_approx]],
    weight=1
)
site_map.add_child(coastline_line_approx)

# Display the map
display(site_map)

Approximate distance to coastline: 0.56 km


In [31]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

# Add a marker for the distance to the approximated coastline point
distance_marker = folium.Marker(
    [coastline_lat, coastline_long_approx],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    )
site_map.add_child(distance_marker)

# Display the map
display(site_map)

In [33]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [[launch_site_lat, launch_site_long], [coastline_lat, coastline_long_approx]]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [34]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

# Example coordinates for a city, railway, and highway near CCAFS LC-40
city_coordinates = [28.6114, -80.8216]  # Titusville, FL
railway_coordinates = [28.5720, -80.5852] # Approximate railway line
highway_coordinates = [28.5626, -80.5707] # Approximate highway I-95

# Calculate distances
distance_to_city = calculate_distance(launch_site_lat, launch_site_long, city_coordinates[0], city_coordinates[1])
distance_to_railway = calculate_distance(launch_site_lat, launch_site_long, railway_coordinates[0], railway_coordinates[1])
distance_to_highway = calculate_distance(launch_site_lat, launch_site_long, highway_coordinates[0], highway_coordinates[1])

# Add markers and lines for city
city_marker = folium.Marker(
    city_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_city),
        )
    )
site_map.add_child(city_marker)
city_line = folium.PolyLine(
    locations=[launch_site_coordinates, city_coordinates],
    weight=1
)
site_map.add_child(city_line)

# Add markers and lines for railway
railway_marker = folium.Marker(
    railway_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_railway),
        )
    )
site_map.add_child(railway_marker)
railway_line = folium.PolyLine(
    locations=[launch_site_coordinates, railway_coordinates],
    weight=1
)
site_map.add_child(railway_line)

# Add markers and lines for highway
highway_marker = folium.Marker(
    highway_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_to_highway),
        )
    )
site_map.add_child(highway_marker)
highway_line = folium.PolyLine(
    locations=[launch_site_coordinates, highway_coordinates],
    weight=1
)
site_map.add_child(highway_line)

# Display the map
display(site_map)